In [1]:
# Dependencies:

#!pip install transformers ipywidgets IPython

# If running on Colab:
#!git clone https://github.com/sam-paech/antislop-sampler.git
#!mv antislop-sampler/src .
#!mv antislop-sampler/slop_phrase_prob_adjustments.json .

In [2]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)
from src.antislop_generate import generate_antislop, chat_antislop

# Enable efficient transfer for Hugging Face models
os.environ['HF_HUB_ENABLE_HF_TRANSFER'] = "1"

# Set the device to 'cuda' if available, else 'cpu'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Specify the model name (replace with your preferred model)
model_name = "unsloth/Llama-3.2-1B-Instruct"
#model_name = "unsloth/Llama-3.2-3B-Instruct"

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, trust_remote_code=True)
model.to(device)
print('Model loaded')

Model loaded


In [3]:
import json
if os.path.exists('banned_tokens.json'):
    with open('banned_tokens.json', 'r') as f:
        slop_phrase_prob_adjustments = dict(json.load(f)[:2000])

In [4]:
prompt = "Write a story about Elara, the weaver of tapestries in future Technopolis. In the bustling city, a group of "

In [ ]:
# Chat generation with streaming
messages = [
    {"role": "user", "content": prompt}
]
tokens = []
text = ''
for token in chat_antislop(
    model=model,
    tokenizer=tokenizer,
    messages=messages,
    max_new_tokens=600,
    # Antislop sampling may be less reliable at low temperatures.
    #temperature=1,    
    #min_p=0.1,
    temperature=0.01,
    # The adjustment_strength param scales how strongly the probability adjustments are applied.
    # A value of 1 means the values in slop_phrase_prob_adjustments (or the defaults) are used unmodified.
    # Reasonable values are 0 (disabled) thru 100+ (effectively banning the list).
    adjustment_strength=100.0,
    # Optional: Provide a list of slop phrases and probability adjustments
    slop_phrase_prob_adjustments=slop_phrase_prob_adjustments,
    enforce_json=False,
    antislop_enabled=True,
    ban_slop_first_tokens=False,
    streaming=True,
    stream_smoothing=True, # On by default; this will smooth out the stutters from backtracking.
):
    tokens.append(token)
    full_text = tokenizer.decode(tokens, skip_special_tokens=True)
    new_text = full_text[len(text):]
    text = full_text
    print(new_text, end='', flush=True)

2375


Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


**The Weaver's Gift**

As the sun rose over the sprawling cityscape, casting its golden glow over the steel skyscrapers, the air was alive with the humongous sounds that filled the air. The city, known by its inhabitants simply by its name, was called the Terraveritas – the City that Never Sleep. Here, the inhabitants lived, worked, played, loved, laughed, argued, debated, argued, debated, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued, argued,